<a href="https://colab.research.google.com/github/guillermodangelo/tesis/blob/master/0_0_data_wrangling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd
import os
# from google.colab import drive

In [0]:
# monta google drive y carga data desde allí
drive.mount("/content/drive")
!ls "drive/My Drive/bases_tesis/"
os.chdir('drive/My Drive/bases_tesis/')
raw_data = pd.read_csv('personas_censo_2011.zip', compression='zip', header=0, sep=',', quotechar='"')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
# carga data de GitHub
raw_data = pd.read_csv('https://github.com/guillermodangelo/tesis/blob/master/personas_censo_2011.zip?raw=true', compression='zip', header=0, sep=',', quotechar='"')

In [0]:
raw_data.head(2)

,DPTO,LOC,SECC,SEGM,PERPH02,PERNA01,PERNA02,PERMI01,PERMI01_1,PERMI01_2,PERMI01_3,PERMI01_4,PERMI02,PERMI02_1,PERMI05,PERMI05_1,PERMI06,PERMI06_1,PERMI06_2,PERMI06_3,PERMI06_4,PERMI07,PERMI07_1,PERMI07_2,PERMI07_3,PERMI07_4
0,1,20,1,1,2,22,1988-11-01,1,NaN,NaN,NaN,0,0,0,1,0,0,NaN,NaN,NaN,0,0,NaN,NaN,NaN,0
1,1,20,1,1,2,84,1927-07-01,1,NaN,NaN,NaN,0,0,0,1,0,0,NaN,NaN,NaN,0,0,NaN,NaN,NaN,0


In [0]:
# toma muestra del 5%
# p = raw_data.sample(frac=0.05, random_state=1)
raw_data.shape

(3285877, 26)

In [0]:
# filtra por la variable PERMI06: lugar de residencia anterior
# 1 = En otra localidad o paraje de este departamento
# 2 = En otro departamento
# 3 = En otro país

# p_migr = raw_data[(raw_data.PERMI06 > 0) & (raw_data.PERMI06 <= 3)]
# print(p_migr.shape)

# p_migr.PERMI06.value_counts()

In [0]:
# filtra por la variable PERMI07: lugar de residencia 5 años antes
# 2 = En otra localidad o paraje de este departamento
# 3 = En otro departamento

p_migr = raw_data[(raw_data.PERMI07 == 2) | (raw_data.PERMI07 == 3)]

In [18]:
print(p_migr.shape)

print(p_migr.PERMI07.value_counts())

p_migr.head()


(215899, 27)
3    148759
2     67140
Name: PERMI07, dtype: int64


,DPTO,LOC,SECC,SEGM,PERPH02,PERNA01,PERNA02,PERMI01,PERMI01_1,PERMI01_2,PERMI01_3,PERMI01_4,PERMI02,PERMI02_1,PERMI05,PERMI05_1,PERMI06,PERMI06_1,PERMI06_2,PERMI06_3,PERMI06_4,PERMI07,PERMI07_1,PERMI07_2,PERMI07_3,PERMI07_4,resid_anterior
5,1,020,1,1,1,24,1987-05-01,1,NaN,NaN,NaN,0,0,0,2,2,2,NaN,3.0,732.0,0,3,NaN,3.0,732.0,0,732
40,1,020,1,1,2,22,1988-11-01,3,NaN,5.0,422.0,0,0,0,2,4,2,NaN,17.0,220.0,0,3,NaN,17.0,220.0,0,220
50,1,020,1,1,1,19,1992-03-01,3,NaN,18.0,220.0,0,0,0,2,1,2,NaN,18.0,220.0,0,3,NaN,18.0,220.0,0,220
64,1,020,1,1,1,24,1986-12-01,3,NaN,2.0,220.0,0,0,0,2,2,2,NaN,2.0,220.0,0,3,NaN,2.0,220.0,0,220
110,1,020,1,1,2,28,1982-11-01,3,NaN,12.0,320.0,0,0,0,2,3,2,NaN,12.0,320.0,0,3,NaN,12.0,320.0,0,320


In [9]:
# generar variable con los códigos de localidad de todos aquellos que vivian en otro lado 5 años antes
p_migr['resid_anterior'] = p_migr.loc[:,('PERMI07_1')]
p_migr.resid_anterior.fillna(p_migr.PERMI07_3, inplace=True)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.6/dist-packages/pandas/core/generic.py:6245: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._update_inplace(new_data)


In [16]:
# cuenta NaNs, debería dar cero
sum(pd.isnull(p_migr['resid_anterior']))

0

In [17]:
# cambia cóódigos de localidad a string de 3 dígitos
p_migr.loc[:,('LOC')] = p_migr.loc[:,('LOC')].astype(str).str.zfill(3)

p_migr.loc[:,('resid_anterior')] = p_migr.loc[:,('resid_anterior')].astype(int).astype(str).str.zfill(3)

# faltaria ver como agregar codigos de departamentos a esto

/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


In [0]:
# genera codlos
p_migr['CODLOC'] =  p_migr['DPTO'].astype(str) + p_migr['LOC']
p_migr['PERMI07_1_CODLOC'] =  p_migr['DPTO'].astype(str) + p_migr['PERMI07_1']
p_migr['PERMI07_3_CODLOC'] =  p_migr['PERMI07_2'].astype(str) + p_migr['PERMI07_3']

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

In [0]:
p_migr.head(2)

,DPTO,LOC,SECC,SEGM,PERPH02,PERNA01,PERNA02,PERMI01,PERMI01_1,PERMI01_2,PERMI01_3,PERMI01_4,PERMI02,PERMI02_1,PERMI05,PERMI05_1,PERMI06,PERMI06_1,PERMI06_2,PERMI06_3,PERMI06_4,PERMI07,PERMI07_1,PERMI07_2,PERMI07_3,PERMI07_4,CODLOC,PERMI07_1_CODLOC,PERMI07_3_CODLOC
5,1,020,1,1,1,24,1987-05-01,1,NaN,NaN,NaN,0,0,0,2,2,2,NaN,3.0,732.0,0,3,nan,3.0,732.0,0,1020,1nan,3.0732.0
40,1,020,1,1,2,22,1988-11-01,3,NaN,5.0,422.0,0,0,0,2,4,2,NaN,17.0,220.0,0,3,nan,17.0,220.0,0,1020,1nan,17.0220.0
